In [15]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

# Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI

In [16]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

list_experiments = client.search_experiments()

for experiment in list_experiments:
    print(f"Name: {experiment.name}, Id: {experiment.experiment_id}")

Name: MLflow Quickstart, Id: 3
Name: ./nyc-taxi-expertiment, Id: 2
Name: nyc-taxi-expertiment, Id: 1
Name: Default, Id: 0


Let's check the latest versions for the experiment with id 1...

In [28]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [75]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")
    #print(run)

run id: de489aa66f6b40879e8ce7807aa65d57, rmse: 124.7081
run id: 4947e66f0f744cc8b6f163cdfcd450c4, rmse: 124.7081
run id: f034660a4f4c4b5fb656d9e1299eaa4e, rmse: 124.7081
run id: 5a6af9494cc3433988badb4b314eab23, rmse: 124.7081
run id: ce3a021a2aa348caafba94c32c75cf8b, rmse: 147.4581


In [77]:
# get the best model ID
best_model_id = runs[0].info.run_id

# Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the nre best experiment nyc-taxi-regressor


In [98]:
# List all registered models
registered_models = client.search_registered_models()

# Print out model names
for model in registered_models:
    print(f"Model Name: {model.name}")
    
model_name = "nyc-taxi-regressor"

Model Name: nyc-taxi-regressor
Model Name: sk-learn-random-forest-reg-model


In [94]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [95]:
run_id = best_model_id
model_uri = f"runs:/{run_id}/model"
model_info = mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


In [99]:
# Get the version of the new best model
model_version = model_info.version

In [ ]:
# Set model version tag
# Seting this new model to validation pending
client.set_model_version_tag(model_name, model_version, "validation_status", "pending")
# Seting a alias to this model champion to show that is the best from this experiment
client.set_registered_model_alias(model_name, "champion", model_version)


In [85]:
# Retrieving information about the model by alias
client.get_model_version_by_alias(model_name, "champion")

<ModelVersion: aliases=['champion', 'first', 'second'], creation_timestamp=1725191021012, current_stage='None', description=None, last_updated_timestamp=1725191021012, name='nyc-taxi-regressor', run_id='f2f44604abb84796a004e6a3f23031a5', run_link=None, source='/workspaces/mlops/02-experiment-tracking/mlruns/1/f2f44604abb84796a004e6a3f23031a5/artifacts/model', status='READY', status_message=None, tags={'validation_status': 'approved'}, user_id=None, version=1>

In [ ]:
def print_registered_model_info(rm):
    print(f"name: {rm.name}")
    print(f"tags: {rm.tags}")
    print(f"description: {rm.description}")

In [ ]:
print_registered_model_info(client.get_registered_model(name))


In [ ]:
client.set_registered_model_alias("example-model", "Champion", 2)
